In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from kret_studies import *
from kret_studies.notebook import *
from kret_studies.complex import *

logger = get_notebook_logger()

Loaded environment variables from /Users/Akseldkw/coding/kretsinger/.env.


INFO:datasets:JAX version 0.7.2 available.
/Users/Akseldkw/micromamba/envs/kret_312/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/Users/Akseldkw/micromamba/envs/kret_312/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated

/Users/Akseldkw/coding/kretsinger/data/nb_log.log


In [ ]:
run = start_wandb_run("mse_example")

wandb: Currently logged in as: akseldkw (akseldkw07) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
handle = "yasserh/housing-prices-dataset"
file_path = "Housing.csv"

df: pd.DataFrame = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, handle, file_path)

In [5]:
uks_pd.data_cleanup(df)
df = uks_pd.one_hot_encode(df)
df = uks_pd.move_columns(df, end=["price"])
X, y = uks_pd.split_x_y(df, "price")

/Users/Akseldkw/coding/kretsinger/kret_studies/kret_pandas.py:139: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(non_null, errors="coerce")


In [6]:
from kret_studies.kret_torch.nn_mse_ce import LinearNN


class HousingNN(LinearNN):
    def set_model(self, input_dim: int):
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

In [7]:
model = HousingNN()
model.set_model(input_dim=X.shape[1])
model.post_init()

[2025-11-13 10:36:52] [ERROR] Failed to load state from /Users/Akseldkw/coding/kretsinger/data/pytorch/HousingNN_v000: [Errno 2] No such file or directory: '/Users/Akseldkw/coding/kretsinger/data/pytorch/HousingNN_v000/weights.pt'. Continuing with fresh weights.


{'hparams': {'batchsize': 128,
             'gamma': 0.1,
             'improvement_tol': 0.0001,
             'lr': 0.001,
             'patience': 25,
             'stepsize': 7},
 'state': {'best_loss': inf, 'epochs_trained': 0}}


In [14]:
model.train_model((X, y), (X, y), epochs=2_500)

100%|██████████| 2500/2500 [01:14<00:00, 33.45it/s]


In [15]:
X.dtypes

area                               int64
bedrooms                           int64
bathrooms                          int64
stories                            int64
mainroad                            bool
guestroom                           bool
basement                            bool
hotwaterheating                     bool
airconditioning                     bool
parking                            int64
prefarea                            bool
furnishingstatus_semi-furnished     bool
furnishingstatus_unfurnished        bool
dtype: object

In [16]:
y.dtypes

dtype('int64')

In [17]:
y

0      13300000
1      12250000
2      12250000
3      12215000
4      11410000
         ...   
540     1820000
541     1767150
542     1750000
543     1750000
544     1750000
Name: price, Length: 545, dtype: int64

In [18]:
# raise Exception("stop here")
run.finish()

Train Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,3032424326187.215
Validation Loss,3032821186229.314
